<a href="https://colab.research.google.com/github/rodolphorosa/intelie-data-science/blob/master/Autoencoder_for_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autoencoder for Collaborative Filtering

In this work, the aim is to design a user-based autoencoder which can predict users' ratings based on their historical data. Each user is represented by a partially observed (that is, sparse) vector whose values correspond to their previous ratings. So, the three layer autoencoder takes a user vector as input, projects it to a low-dimensional space (hidden layer) and then reconstructs it in the output layer, so that the missing values are filled. The model proposed is based on the works of Suvash Sedhain et al. (``AutoRec: Autoencoders Meet Collaborative Filtering``) and Kuchaiev & Ginsburg (``Training Deep AutoEncoders for Collaborative Filtering``). 

ReLu (Rectified Linear Unit) was chosen as activation (transfer) function of the input layer. Although sigmoid and hyperbolic tangent are widely used as activation function in many machine learning problemns and can , they could not certainly work so well in this scenario. First of all, the output they generate are bounded by a very tiny interval (\[0, 1\] for sigmoid and \[-1, 1\]) for hyperbolic tangent. This is bad firstly because the possibility of learning zero ratings, which is wanted to be avoided, and for the fact that the tangent function may generate negative values, which is even worse. Besides, both functions are prone to the vanishing gradient problem, which prevents the update of the weights of the learning algorithm. Despite ReLu turns negative input into zero, this functions does not have superior limit, which is more adequate to this problem. Forthermore, ReLu overcomes the vanishing learning problem, then the algorithm learns faster. The formula of ReLu is the following:

$$ReLu=max\{0, x\}$$

Due to the nature of the given problem, a linear function was chosen as transfer function between the hidden and the output layer. Once our problem is not probabilistic, sigmoid or softmax are not good choices once they describe, respectively, logistic and probabilistic distribution. Besides, they are both bounded.

Mean absolute error as chosen as loss function. This way, the proposed model tries to minimize the average difference between the actual ratings and the predicted ones. Once, predicting zeros is not wanted, a custom version of this function is used. In this version, only the difference between observed ratings and their corresponding outputs is calculated. Thus, it is garanteed that missing values will be learnt. The formula is the following: 

$$MAE=\sum_{i=0}^{n}\frac{|r_{i} - y_{i}|}{n}$$

The number of hidden unit were chosen based on the results achieve by the previosu works. This way, the size of hidden layer of this model is of 128, which is good to avoid overfitting during the training. The weights of the neural network are learnt by stochastic gradient descent algorithm, learning rate of 0.001 and momentum of 0.9.

To evaluate the performance of the model, k-fold cross-validation was used, with $ k = 5 $. The rating matrix was divided into five disjoined sets of training and test data. 80% of the input was used as training, while 20% was used for testing. The number of epochs in each experiment is 100. 

The implementation of the autoencoder was done by using Python's machine learning library Keras. 

## Implementation

Keras was used to implement the core of this approach, which is autoencoder itself. Scikit Learn library provides the tools for the cross validation, in this case, K-fold. Finally, Scipy and Numpy were used for data representation. 

In [0]:
import tensorflow as tf 

from keras.models import Sequential 
from keras.layers import Dense 
from keras import backend as K, optimizers 
from sklearn.model_selection import KFold 
from scipy import sparse 

import numpy as np 

Using TensorFlow backend.


The following method implements the Mean Squared Error. In this work, its formula has been modified so that only the error between observed ratings and their corresponding output in the reconstructed vector is measured. This prevents the algorithm to learn zeros. 

In [0]:
def mae(y_true, y_pred):
    return K.mean(K.abs(y_true[y_true > 0] - y_pred[y_true > 0]))

A rating matrix R is then built with dimension (6040, 3952). Each line comprises a user, whereas each column represents a movie. Once the number of observed ratings is very small compared to the size of the matrix (only 4% of the matrix is filled), it is then converted into a sparse matrix, what significantly reduces the use of memory.

In [0]:
n_users = 6040
n_movies = 3952

R = np.zeros((n_users, n_movies))

with open("ratings.dat") as file:
    for line in file.readlines():
        user_id, movie_id, rating, timestamp = line.split("::")
        R[int(user_id) - 1, int(movie_id) - 1] = rating

R = sparse.csr_matrix(R)

The model is then constructed as a sequence of dense (fully connected) layers. The transfer function from the input to the hidden layer is ReLu, while the hidden layer has a linear function as activaton. The loss function is the aforementioned custom MAE.

In [0]:
model = Sequential()
model.add(Dense(128, input_dim=3952, activation="relu", use_bias=True))
model.add(Dense(3952, activation="linear"))
sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(loss=mae, optimizer=sgd)

The sparse rating matrix R is then divided into 5 disjoined sets of training and test data. For each pair of training and test sets, the autoencoder learns the ratings from the training set and then the quality of the learning is evaluated with the test set. 

In [0]:
kf = KFold(n_splits=5, random_state=None, shuffle=False)

evaluations = []

for train, test in kf.split(R):
    errors = []
    
    X_train = R[train]
    X_test = R[test]
    
    model.fit(X_train, X_train, epochs=100, verbose=2)
    
    evaluations.append(model.evaluate(X_test, X_test))

Epoch 1/100
 - 2s - loss: 3.5365
Epoch 2/100
 - 2s - loss: 3.0122
Epoch 3/100
 - 2s - loss: 2.1498
Epoch 4/100
 - 2s - loss: 1.8543
Epoch 5/100
 - 2s - loss: 1.7199
Epoch 6/100
 - 2s - loss: 1.6319
Epoch 7/100
 - 2s - loss: 1.5610
Epoch 8/100
 - 2s - loss: 1.5072
Epoch 9/100
 - 2s - loss: 1.4571
Epoch 10/100
 - 2s - loss: 1.4155
Epoch 11/100
 - 2s - loss: 1.3922
Epoch 12/100
 - 2s - loss: 1.3529
Epoch 13/100
 - 2s - loss: 1.3377
Epoch 14/100
 - 2s - loss: 1.3170
Epoch 15/100
 - 2s - loss: 1.2950
Epoch 16/100
 - 2s - loss: 1.2762
Epoch 17/100
 - 2s - loss: 1.2646
Epoch 18/100
 - 2s - loss: 1.2476
Epoch 19/100
 - 2s - loss: 1.2276
Epoch 20/100
 - 2s - loss: 1.2199
Epoch 21/100
 - 2s - loss: 1.2075
Epoch 22/100
 - 2s - loss: 1.1938
Epoch 23/100
 - 2s - loss: 1.1910
Epoch 24/100
 - 2s - loss: 1.1750
Epoch 25/100
 - 2s - loss: 1.1679
Epoch 26/100
 - 2s - loss: 1.1578
Epoch 27/100
 - 2s - loss: 1.1513
Epoch 28/100
 - 2s - loss: 1.1431
Epoch 29/100
 - 2s - loss: 1.1334
Epoch 30/100
 - 2s - lo

 - 2s - loss: 0.7360
Epoch 40/100
 - 2s - loss: 0.7352
Epoch 41/100
 - 2s - loss: 0.7345
Epoch 42/100
 - 2s - loss: 0.7339
Epoch 43/100
 - 2s - loss: 0.7323
Epoch 44/100
 - 2s - loss: 0.7330
Epoch 45/100
 - 2s - loss: 0.7296
Epoch 46/100
 - 2s - loss: 0.7304
Epoch 47/100
 - 2s - loss: 0.7288
Epoch 48/100
 - 2s - loss: 0.7283
Epoch 49/100
 - 2s - loss: 0.7272
Epoch 50/100
 - 2s - loss: 0.7264
Epoch 51/100
 - 2s - loss: 0.7256
Epoch 52/100
 - 2s - loss: 0.7254
Epoch 53/100
 - 2s - loss: 0.7236
Epoch 54/100
 - 2s - loss: 0.7234
Epoch 55/100
 - 2s - loss: 0.7227
Epoch 56/100
 - 2s - loss: 0.7229
Epoch 57/100
 - 2s - loss: 0.7223
Epoch 58/100
 - 2s - loss: 0.7201
Epoch 59/100
 - 2s - loss: 0.7201
Epoch 60/100
 - 2s - loss: 0.7178
Epoch 61/100
 - 2s - loss: 0.7184
Epoch 62/100
 - 2s - loss: 0.7170
Epoch 63/100
 - 2s - loss: 0.7165
Epoch 64/100
 - 2s - loss: 0.7158
Epoch 65/100
 - 2s - loss: 0.7156
Epoch 66/100
 - 2s - loss: 0.7167
Epoch 67/100
 - 2s - loss: 0.7137
Epoch 68/100
 - 2s - loss: 

 - 2s - loss: 0.6288
Epoch 78/100
 - 2s - loss: 0.6278
Epoch 79/100
 - 2s - loss: 0.6269
Epoch 80/100
 - 2s - loss: 0.6269
Epoch 81/100
 - 2s - loss: 0.6262
Epoch 82/100
 - 2s - loss: 0.6273
Epoch 83/100
 - 2s - loss: 0.6269
Epoch 84/100
 - 2s - loss: 0.6266
Epoch 85/100
 - 2s - loss: 0.6253
Epoch 86/100
 - 2s - loss: 0.6248
Epoch 87/100
 - 2s - loss: 0.6249
Epoch 88/100
 - 2s - loss: 0.6245
Epoch 89/100
 - 2s - loss: 0.6253
Epoch 90/100
 - 2s - loss: 0.6241
Epoch 91/100
 - 2s - loss: 0.6244
Epoch 92/100
 - 2s - loss: 0.6233
Epoch 93/100
 - 2s - loss: 0.6239
Epoch 94/100
 - 2s - loss: 0.6222
Epoch 95/100
 - 2s - loss: 0.6209
Epoch 96/100
 - 2s - loss: 0.6212
Epoch 97/100
 - 2s - loss: 0.6217
Epoch 98/100
 - 2s - loss: 0.6219
Epoch 99/100
 - 2s - loss: 0.6212
Epoch 100/100
 - 2s - loss: 0.6205
1208/1208 [==============================] - 0s 151us/step


After the execution of the experiments, the average error is measured. 

In [0]:
np.mean(evaluations)

0.80070180671894

The results showed that the autoencoder has error of approximately 0.80, similarly to the results achieved by previous works. Therefore, the predicted ratings are considerably close to the actual ones so that the autoencoder is likely to understand the taste of the users within a small margin of error. This results indicate that a recommender system based on the proposed model will certainly present to the users films they will probably be interested to see, which is main goal of this approach. 